In [ ]:
from experiment import db_utils as db
import pandas as pd
import json
from typing import Callable, Dict, Sequence

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

# from jupyterthemes import jtplot

# # you can select an alternative theme's plot style by name
# # onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
# jtplot.style('onedork')

In [ ]:
# data aggregation

def aggregate_by_high_node(rs, col_name: str, aggregate: Callable, scale=1, get_argmax=False):
    labels = ["sentence_q", "subj_adj", "subj_noun", "subj", "neg", "negp", 
              "v_adv", "v_verb", "v_bar", "vp", "vp_q", 
              "obj_adj", "obj_noun", "obj"]
    labels2idx = {l: i for i, l in enumerate(labels)}
    baseline = [None] * len(labels)
    trained = [None] * len(labels)
    if get_argmax:
        if not aggregate == max:
            raise ValueError("Can only get argmax if aggreate is max")
        argmax = [None] * len(labels)
    
    if aggregate not in {max, min, sum}:
        raise ValueError("Does not support this type of aggregation")
    
    for row in rs:
        abstraction = json.loads(row["abstraction"])
        high_node = abstraction[0]
        low_node = abstraction[1][0]

        is_baseline = "baseline" in row["model_path"]
        i = labels2idx[high_node]
        
        if col_name.endswith("s"):
            values = json.loads(row[col_name])
            value = aggregate(values)
        else:
            value = row[col_name]
        
        value *= scale
        
        if is_baseline:
            if baseline[i] is None:
                baseline[i] = value
                if get_argmax:
                    argmax[i] = low_node[-1]
            else:
                if get_argmax:
                    if value > baseline[i]:
                        baseline[i] = value
                        argmax[i] = low_node[-1]
                else:
                    baseline[i] = aggregate([baseline[i], value])
                
        else:
            if trained[i] is None:
                trained[i] = value
            else:
                trained[i] = aggregate([trained[i], value])
    
    if get_argmax:
        return labels, baseline, trained, argmax
    else:
        return labels, baseline, trained
    
def aggregate_bert_data_by_layer(rs, col_name, high_nodes=None, aggregate=None, scale=1):
    low_nodes = [f"bert_layer_{i}" for i in range(11)]
    low_node_to_idx = {l: i for i, l in enumerate(low_nodes)}
    if high_nodes is None:
        high_nodes = ["sentence_q", "subj_adj", "subj_noun", "subj", "neg", "negp", 
                      "v_adv", "v_verb", "v_bar", "vp", "vp_q", 
                      "obj_adj", "obj_noun", "obj"]
    high_node_to_idx = {l: i for i, l in enumerate(high_nodes)}
    res = [[0] * len(low_nodes) for _ in high_nodes]

    for row in rs:
        abstraction = json.loads(row["abstraction"])
        high_node = abstraction[0]
        low_node = abstraction[1][0]

        if "baseline" in row["model_path"]: continue
        
        idx = high_node_to_idx[high_node]
        if col_name.endswith("s"):
            values = json.loads(row[col_name])
            value = aggregate(values)
        else:
            value = row[col_name]
        
        value *= scale
        
        val_idx = low_node_to_idx[low_node]
        res[idx][val_idx] = value
        
            
    return [f"layer_{i}" for i in range(11)], res, high_nodes

def aggregate_lstm_data(rs, col_name, aggregate=None, scale=1):
    labels = ["sentence_q", "subj_adj", "subj_noun", "subj", "neg", "negp", 
              "v_adv", "v_verb", "v_bar", "vp", "vp_q", 
              "obj_adj", "obj_noun", "obj"]
    labels2idx = {l: i for i, l in enumerate(labels)}
    baseline = [None] * len(labels)
    trained = [None] * len(labels)
    for row in rs:
        abstraction = json.loads(row["abstraction"])
        high_node = abstraction[0]
        is_baseline = "baseline" in row["model_path"]
        i = labels2idx[high_node]
        if col_name.endswith("s"):
            values = json.loads(row[col_name])
            value = aggregate(values)
        else:
            value = row[col_name]
        
        value *= scale
        
        if is_baseline:
            baseline[i] = value if baseline[i] is None else aggregate([baseline[i], value])
        else:
            trained[i] = value if trained[i] is None else aggregate([trained[i], value])
            
    return labels, baseline, trained


def aggregate_data_by_ratio(rows, numerator_cols: Sequence[str], denominator_cols: Sequence[str], aggregate: Callable):
    labels = ["sentence_q", "subj_adj", "subj_noun", "subj", "neg", "negp", 
              "v_adv", "v_verb", "v_bar", "vp", "vp_q", 
              "obj_adj", "obj_noun", "obj"]
    labels2idx = {l: i for i, l in enumerate(labels)}
    res = [None] * len(labels)
    
    if aggregate not in {max, min, sum}:
        raise ValueError("Does not support this type of aggregation")
    
    for row in rows:
        abstraction = json.loads(row["abstraction"])
        high_node = abstraction[0]

        i = labels2idx[high_node]

        numerator_values = sum(np.array(json.loads(row[c])) for c in numerator_cols)
        denominator_values = sum(np.array(json.loads(row[c])) for c in denominator_cols)
        values = [x / y if y != 0 else 0 for x, y in zip(numerator_values, denominator_values)]
        value = aggregate(values)

        
        if res[i] is None:
            res[i] = value
        else:
            res[i] = aggregate([res[i], value])
    
    return labels, res

def max_data_by_normalized_columns(rows, cols, maximizing_cols):
    labels = ["sentence_q", "subj_adj", "subj_noun", "subj", "neg", "negp", 
              "v_adv", "v_verb", "v_bar", "vp", "vp_q", 
              "obj_adj", "obj_noun", "obj"]
    labels2idx = {l: i for i, l in enumerate(labels)}
    cols2idx = {c: i for i, c in enumerate(cols)}
    res = [None] * len(labels)

    for row in rows:
        abstraction = json.loads(row["abstraction"])
        high_node = abstraction[0]
        i = labels2idx[high_node]
        
        totals = sum(np.array(json.loads(row[c])) for c in cols)
        ratios = [np.array(json.loads(row[c])) / totals for c in cols]
        
        # print("totals", totals)
        # print("ratios", ratios)
        
        max_vals = sum(ratios[cols2idx[c]] for c in maximizing_cols)
        argmax_i = np.argmax(max_vals)
        # print("max_vals", max_vals)
        # print("argmax_i", argmax_i)

        max_val = max_vals[argmax_i]
        res_ratios = np.array([r[argmax_i] for r in ratios])
        
        if res[i] is None:
            res[i] = res_ratios
        else:
            orig_max_val = sum(res[i][cols2idx[c]] for c in maximizing_cols)
            if max_val > orig_max_val:
                res[i] = res_ratios
        
    res = [np.array([r[c] for r in res]) for c in range(len(cols))]
    return labels, res
    



In [ ]:
# tools for plotting
def multi_line_plot(x_labels: Sequence[str], 
                    data: Sequence[Sequence],
                    line_labels: Sequence[str],
                    y_label: str, 
                    title: str):
    
    xs = np.arange(len(x_labels))  # the label locations
    assert len(data) == len(line_labels)

    fig, ax = plt.subplots(figsize=(8, 7))
    
    lines = []
    for i, (ys, label) in enumerate(zip(data, line_labels)):
        line = ax.plot(xs, ys, label=label)
        lines.append(line)
    

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel(y_label)
    ax.set_title(title)
    ax.set_xticks(xs)
    ax.set_xticklabels(x_labels)
    ax.legend()

    fig.tight_layout()

    plt.show()
    

    
def multi_bar_plot(x_labels: Sequence[str], 
                   data: Sequence[Sequence],
                   category_labels: Sequence[str],
                   bar_y_label: str, 
                   title: str, 
                   category_styles: Dict={},
                   line_y_label: str=None,
                   width: int=0.7, 
                   bar_label_fmt: str='{:.1%}'):
    
    xs = np.arange(len(x_labels))  # the label locations
    assert len(data) == len(category_labels)

    bar_labels = [k for k in category_labels if category_styles.get(k, "bar") == "bar"]
    line_labels = [k for k in category_labels if k in category_styles and category_styles[k] == "line"]
    
    bar_data = [data for data, label in zip(data, category_labels) if label in bar_labels]
    line_data = [data for data, label in zip(data, category_labels) if label in line_labels]

    bar_width = width / len(bar_labels)

    fig, ax = plt.subplots(figsize=(12, 5))
    
    bars = []
    for i, (ys, label) in enumerate(zip(bar_data, bar_labels)):
        bar_positions = xs - width/2 + bar_width * i
        bar = ax.bar(bar_positions, ys, bar_width, label=label, align="edge")
        bars.append(bar)
    
    if not line_y_label:
        lines = []
        for i, (ys, label) in enumerate(zip(line_data, line_labels)):
            color_id = len(bars) + i
            line = ax.plot(xs, ys, color=f"C{color_id}", label=label)

        # Add some text for labels, title and custom x-axis tick labels, etc.
        ax.set_ylabel(bar_y_label)
        ax.set_title(title)
        ax.set_xticks(xs)
        ax.set_xticklabels(x_labels)
        ax.legend()
    else:
        ax.set_ylabel(bar_y_label)
        ax.set_title(title)
        ax.set_xticks(xs)
        ax.set_xticklabels(x_labels)
        ax.legend()
        
        ax2 = ax.twinx()
        lines = []
        for i, (ys, label) in enumerate(zip(line_data, line_labels)):
            color_id = len(bars) + i
            line = ax2.plot(xs, ys, color=f"C{color_id}", label=label)
        y_axis_color = f"C{len(bars)}"
        ax2.set_ylabel(line_y_label, color=y_axis_color)
        ax2.tick_params(axis='y', labelcolor=y_axis_color)
        
        ax2.legend()


    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            height = rect.get_height()
            ax.annotate(bar_label_fmt.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')
    
    for bar in bars:
        autolabel(bar)

    fig.tight_layout()

    plt.show()
    
def stacked_bar_plot(x_labels: Sequence[str], 
                       data: Sequence[Sequence],
                       category_labels: Sequence[str],
                       bar_y_label: str, 
                       title: str, 
                       bar_width: int=0.5, 
                       bar_label_fmt: str='{:.1%}'):
    xs = np.arange(len(x_labels))
    
    fig, ax = plt.subplots(figsize=(12, 5))
    bars = []
    bottom = np.array([0.0 for _ in xs])
    for i, (ys, label) in enumerate(zip(data, category_labels)):
        bar = ax.bar(xs, ys, bar_width, bottom=bottom, label=label)
        bars.append(bar)
        bottom += np.array(ys)
    
    ax.set_ylabel(bar_y_label)
    ax.set_title(title)
    ax.set_xticks(xs)
    ax.set_xticklabels(x_labels)
    ax.legend()
    
    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            x = rect.get_x() + rect.get_width() / 2
            height = rect.get_height()
            y = rect.get_y() + height
            
            ax.annotate(bar_label_fmt.format(height),
                        xy=(x, y),
                        xytext=(0, -13),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')
    
    for bar in bars:
        autolabel(bar)

    fig.tight_layout()
    
def multi_stacked_bar_plot(x_labels: Sequence[str],
                           data: Sequence[Sequence],
                           series_labels: Sequence[str],
                           category_labels: Sequence[str],
                           bar_y_label: str, 
                           title: str, 
                           bar_width: int=0.7, 
                           bar_label_fmt: str='{:.1%}'):
    x_range = np.arange(len(x_labels))
    one_bar_width = bar_width / len(data)
    
    fig, ax = plt.subplots(figsize=(12, 5))
    bars = []
    
    if len(category_labels) == 5:
        colors = ["#5dade2",  "#E74C3C", "#85C1E9", "#F1948A", "#A6ACAF"]
    elif
        

    for i, (series_data, series_label) in enumerate(zip(data, series_labels)):
        xs = x_range - bar_width/2 + one_bar_width * i
        bottom = np.array([0.0 for _ in xs])
        for j, (ys, category_label) in enumerate(zip(series_data, category_labels)):
            if len(category_labels) == 5:
                hatch = "///" if j == 0 or j == 1 else ""
            else:
                hatch = ""
                
            if i == 0:
                bar = ax.bar(xs, ys, one_bar_width-0.05, bottom=bottom, label=category_label, align="edge", hatch=hatch, color=colors[j])
            else:
                bar = ax.bar(xs, ys, one_bar_width-0.05, bottom=bottom, align="edge", hatch=hatch, color=colors[j])
            bars.append(bar)
            bottom += np.array(ys)
    
    ax.set_ylabel(bar_y_label)
    ax.set_title(title)
    ax.set_xticks(x_range)
    ax.set_xticklabels(x_labels)
    ax.legend()
    
    def autolabel(rects):
        """Attach a text label above each bar in *rects*, displaying its height."""
        for rect in rects:
            x = rect.get_x() + rect.get_width() / 2
            height = rect.get_height()
            y = rect.get_y() + height
            
            ax.annotate(bar_label_fmt.format(height),
                        xy=(x, y),
                        xytext=(0, -13),  # 3 points vertical offset
                        textcoords="offset points",
                        ha='center', va='bottom')
    
    if bar_label_fmt:
        for bar in bars:
            autolabel(bar)

    fig.tight_layout()
    


In [ ]:
bert_hard_data = db.select("experiment_data/bert/bert-hard.db", "results", cond_dict={"status": 3})
print(f"got {len(bert_hard_data)} rows from bert_hard_data")
lstm_hard_data = db.select("experiment_data/lstm/lstm-hard.db", "results", cond_dict={"status": 3})
print(f"got {len(lstm_hard_data)} rows from lstm_hard_data")

bert_easy_data = db.select("experiment_data/bert/bert-easy.db", "results", cond_dict={"status": 3})
print(f"got {len(bert_easy_data)} rows from bert_easy_data")
lstm_easy_data = db.select("experiment_data/lstm/lstm-easy.db", "results", cond_dict={"status": 1})
print(f"got {len(lstm_easy_data)} rows from lstm_easy_data")

bert_baseline_data = db.select("experiment_data/bert/bert-baseline.db", "results", cond_dict={"status": 1})
lstm_baseline_data = db.select("experiment_data/lstm/lstm-baseline.db", "results", cond_dict={"status": 2})

# lstm_hard_data = db.select("experiment_data/sep/nov2-500.db", "results", cond_dict={"status": 4}, like={"abstraction": "%lstm_0%"})
# print(len(old_lstm_easy_data))


<img src="figures/intervention.png">

$$\text{weight}(x, x', V) = \text{Count}^{-1}[V' : y_{V' \leftarrow V'(x')}(x) = y_{V \leftarrow V(x')} (x)] $$

## Schema of interchange experiment

<img src="figures/interchange_experiment.png" >

## Plot by outcome category

<img src="figures/outcome_categories_colored.png" >

In [ ]:
# def get_normalized_column_data(data):
#     all_cols = [f"res_{i}_counts" for i in range(8)]
#     maximizing_cols = ["res_3_counts"]

#     labels, res = max_data_by_normalized_columns(data, all_cols, maximizing_cols)

#     non_causal = sum(res[4:8])

#     data = res[:4] + [non_causal]
#     category_labels = ["!base, !success", "!base, success", "base, !success", "base, success", "non-causal"]
#     return labels, data, category_labels


def get_normalized_column_data(data):
    all_cols = [f"res_{i}_counts" for i in range(8)]
    maximizing_cols = ["res_3_counts"]

    labels, res = max_data_by_normalized_columns(data, all_cols, maximizing_cols)
    

    incorrect = sum([res[0], res[1], res[4], res[5]])

    data = [res[3], res[2], res[7],  res[6], incorrect]
    category_labels = ["causal, successful",  "causal, unsuccessful", "non-causal, successful",  "non-causal, unsuccessful", "base result incorrect"]
    return labels, data, category_labels

In [ ]:
labels, bert_category_data, category_labels = get_normalized_column_data(bert_hard_data)
_, lstm_category_data, _ = get_normalized_column_data(lstm_hard_data)
_, lstm_baseline_category_data, _ = get_normalized_column_data(lstm_baseline_data)
_, bert_baseline_category_data, _ = get_normalized_column_data(bert_baseline_data)
print(bert_baseline_category_data)
print(category_labels)

# stacked_bar_plot(labels, bert_category_data, category_labels, "ratio", "test")
multi_stacked_bar_plot(labels, [bert_baseline_category_data, lstm_category_data, bert_category_data], 
                       ["bert_baseline", "lstm", "bert"], 
                       category_labels, 
                       "ratio", "Breakdown of outcomes of models trained on hard datasets (left to right: LSTM baseline, BERT baseline, LSTM, BERT)", 
                       bar_width=0.7,
                       bar_label_fmt="")

In [ ]:
labels, bert_category_data, category_labels = get_normalized_column_data(bert_easy_data)
_, lstm_category_data, category_labels = get_normalized_column_data(lstm_easy_data)
_, lstm_baseline_category_data, _ = get_normalized_column_data(lstm_baseline_data)


# stacked_bar_plot(labels, bert_category_data, category_labels, "ratio", "test")
multi_stacked_bar_plot(labels, [lstm_baseline_category_data, lstm_category_data, bert_category_data], 
                       ["baseline", "lstm", "bert"], 
                       category_labels, 
                       "ratio", "Breakdown of outcomes of models trained on easy datasets (left to right: Baseline, LSTM, BERT)", 
                       bar_width=0.7,
                       bar_label_fmt="")

In [ ]:
def get_normalized_causal_column_data(data):
    all_cols = [f"res_{i}_counts" for i in range(8)]
    maximizing_cols = ["res_3_counts"]

    labels, res = max_data_by_normalized_columns(data, all_cols, maximizing_cols)

    total = sum(res[:4])
    
    data = [r / total for r in res]

    category_labels = ["!base_eq, !success", "!base_eq, success", "base_eq, !success", "base_eq, success"]
    return labels, data, category_labels

In [ ]:
labels, bert_category_data, category_labels = get_normalized_causal_column_data(bert_hard_data)
_, lstm_category_data, category_labels = get_normalized_causal_column_data(lstm_hard_data)
_, baseline_category_data, _ = get_normalized_causal_column_data(lstm_baseline_data)


# stacked_bar_plot(labels, bert_category_data, category_labels, "ratio", "test")
multi_stacked_bar_plot(labels, [baseline_category_data, lstm_category_data, bert_category_data], 
                       ["baseline", "lstm", "bert"],
                       category_labels, 
                       "ratio", "Breakdown of CAUSAL outcomes of models trained on hard datasets (left to right: Baseline, LSTM, BERT)", bar_width=0.7, bar_label_fmt="")

In [ ]:
labels, bert_category_data, category_labels = get_normalized_causal_column_data(bert_easy_data)
_, lstm_category_data, category_labels = get_normalized_causal_column_data(lstm_easy_data)
_, baseline_category_data, _ = get_normalized_causal_column_data(lstm_baseline_data)

# stacked_bar_plot(labels, bert_category_data, category_labels, "ratio", "test")
# stacked_bar_plot(labels, bert_category_data, category_labels, "ratio", "test")
multi_stacked_bar_plot(labels, [baseline_category_data, lstm_category_data, bert_category_data], 
                       ["baseline", "lstm", "bert"],
                       category_labels, 
                       "ratio", "Breakdown of CAUSAL outcomes of models trained on hard datasets (left to right: Baseline, LSTM, BERT)", 
                       bar_width=0.7, bar_label_fmt="")

## Maximum Intervention Success Rates

$$\text{Success rate} = \frac{Count[\text{intervention correct, low base output correct, intervention impactful}]}{Count[\text{low base output correct, intervention impactful}]} $$

In [ ]:
def plot_max_success_rate(baseline_data, lstm_data, bert_data, title):
    numerator_cols = ["res_3_counts"]
    denominator_cols = ["res_2_counts", "res_3_counts"]

    labels, bert_hard_success_rates = aggregate_data_by_ratio(bert_data, numerator_cols, denominator_cols, max)
    _, lstm_hard_success_rates = aggregate_data_by_ratio(lstm_data, numerator_cols, denominator_cols, max)
    _, lstm_baseline_success_rates = aggregate_data_by_ratio(baseline_data, numerator_cols, denominator_cols, max)

    data_for_plot = [lstm_hard_success_rates, bert_hard_success_rates]
    data_labels = ["LSTM", "Bert"]
    multi_bar_plot(labels, data_for_plot, data_labels, bar_y_label="success rate", title=title)

In [ ]:
plot_max_success_rate(lstm_baseline_data, lstm_easy_data, bert_easy_data, "Intervention success rate on correct inputs and causal interventions, models trained on easy dataset")

In [ ]:
plot_max_success_rate(lstm_baseline_data, lstm_hard_data, bert_hard_data, "Intervention success rate on correct inputs and causal interventions, models trained on easy dataset")

## Relaxed success rate

$$\text{Relaxed success rate} = \frac{Count[\text{intervention correct, intervention impactful}]}{Count[\text{intervention impactful}]} $$

In [ ]:
def plot_relaxed_success_rate(baseline_data, lstm_data, bert_data, title):
    numerator_cols = ["res_1_counts", "res_3_counts"]
    denominator_cols = ["res_0_counts", "res_1_counts", "res_2_counts", "res_3_counts"]
    labels, bert_success_rates = aggregate_data_by_ratio(bert_data, numerator_cols, denominator_cols, max)
    _, lstm_success_rates = aggregate_data_by_ratio(lstm_data, numerator_cols, denominator_cols, max)
    _, baseline_success_rates = aggregate_data_by_ratio(baseline_data, numerator_cols, denominator_cols, max)

    data_for_plot = [baseline_success_rates, lstm_success_rates, bert_success_rates]
    data_labels = ["Baseline", "LSTM", "Bert"]
    multi_bar_plot(labels, data_for_plot, data_labels, bar_y_label="relaxed success rate", title=title)

In [ ]:
plot_relaxed_success_rate(lstm_baseline_data, lstm_easy_data, bert_easy_data, "Maximum success rate on all inputs, trained on easy dataset")

In [ ]:
plot_relaxed_success_rate(lstm_baseline_data, lstm_hard_data, bert_hard_data, "Maximum success rate on all inputs, trained on hard dataset")

In [ ]:
def plot_success_rate(baseline_data, lstm_data, bert_data, title):
    numerator_cols = ["res_6_counts"]
    denominator_cols = ["res_6_counts", "res_7_counts"]
    labels, bert_success_rates = aggregate_data_by_ratio(bert_data, numerator_cols, denominator_cols, max)
    _, lstm_success_rates = aggregate_data_by_ratio(lstm_data, numerator_cols, denominator_cols, max)
    _, baseline_success_rates = aggregate_data_by_ratio(baseline_data, numerator_cols, denominator_cols, max)

    data_for_plot = [baseline_success_rates, lstm_success_rates, bert_success_rates]
    data_labels = ["Baseline", "LSTM", "Bert"]
    multi_bar_plot(labels, data_for_plot, data_labels, bar_y_label="relaxed success rate", title=title)

In [ ]:
plot_success_rate(lstm_baseline_data, lstm_easy_data, bert_easy_data, "Maximum success rate on all inputs, trained on easy dataset")

In [ ]:
plot_success_rate(lstm_baseline_data, lstm_hard_data, bert_hard_data, "Maximum success rate on all inputs, trained on easy dataset")

## Error base result success rate
$$\text{Error base result success rate} = \frac{Count[\text{intervention correct, low base result incorrect, intervention impactful}]}{Count[\text{low base result incorrect, intervention impactful}]} $$

In [ ]:
def plot_error_base_result_success_rate(baseline_data, lstm_data, bert_data, title):
    numerator_cols = ["res_1_counts"]
    denominator_cols = ["res_0_counts", "res_1_counts"]
    labels, bert_success_rates = aggregate_data_by_ratio(bert_data, numerator_cols, denominator_cols, max)
    _, lstm_success_rates = aggregate_data_by_ratio(lstm_data, numerator_cols, denominator_cols, max)
    _, baseline_success_rates = aggregate_data_by_ratio(baseline_data, numerator_cols, denominator_cols, max)

    data_for_plot = [baseline_success_rates, lstm_success_rates, bert_success_rates]
    data_labels = ["baseline", "LSTM", "Bert"]
    multi_bar_plot(labels, data_for_plot, data_labels, bar_y_label="error base result success rate", title=title)

In [ ]:
plot_error_base_result_success_rate(lstm_baseline_data, lstm_easy_data, bert_easy_data, "Maximum success rate on inputs wrongly classified by model, trained on easy dataset")

In [ ]:
plot_error_base_result_success_rate(lstm_baseline_data, lstm_hard_data, bert_hard_data, "Maximum success rate on inputs wrongly classified by model, trained on hard dataset")

In [ ]:
# labels, bert_baseline_eq_rate, bert_eq_rate, argmax = aggregate_by_high_node(bert_data, "interv_eq_rates", aggregate=max, get_argmax=True)
# _, lstm_baseline_eq_rate, lstm_eq_rate = arrange_lstm_data(lstm_data, "interv_eq_rates", aggregate=max)
# _, _, causal_ratio = aggregate_by_high_node(bert_data, "effective_ratios", aggregate=max, scale=250000)

# data_for_plot = [bert_baseline_eq_rate, lstm_baseline_eq_rate, lstm_eq_rate, bert_eq_rate, causal_ratio]
# data_labels = ["Bert Baseline", "LSTM Baseline", "LSTM", "Bert", "Causal interchanges"]
# category_styles = {"Causal interchanges": "line"}
# multi_bar_plot(labels, data_for_plot, data_labels, 
#                category_styles=category_styles,
#                bar_y_label="#successful / #causal", 
#                line_y_label="#causal (#total is 250000)",
#                title="Maximum intervention success rate among all low-level locations")

## Success rate by bert layer

In [ ]:
low_nodes, values, high_nodes = aggregate_bert_data_by_layer(bert_data, "interv_eq_rates", high_nodes=None, aggregate=max)
multi_line_plot(low_nodes, values, high_nodes, y_label="#successful / #causal", title="Intervention success rate by bert layer")

## Clique size aggregates

In [ ]:
# Max
labels, _, lstm_clq_size = aggregate_lstm_data(lstm_easy_data, "max_clique_sizes", aggregate=max)
_, _, bert_clq_size = aggregate_by_high_node(bert_easy_data, "max_clique_sizes", aggregate=max)
_, _, lstm_baseline_clq_size = aggregate_lstm_data(lstm_baseline_data, "max_clique_sizes", aggregate=max)

print(lstm_baseline_clq_size)
print(lstm_clq_size)
print(bert_clq_size)

data_for_plot = [lstm_clq_size, bert_clq_size]
data_labels = ["LSTM", "Bert"]
multi_bar_plot(labels, data_for_plot, data_labels, 
               bar_y_label="#successful / #causal", 
               title="Maximum clique sizes for models trained on the hard dataset",
               bar_label_fmt='{}')

In [ ]:
# Max
labels, _, lstm_clq_size = aggregate_lstm_data(lstm_hard_data, "max_clique_sizes", aggregate=max)
_, _, bert_clq_size = aggregate_by_high_node(bert_hard_data, "max_clique_sizes", aggregate=max)
_, _, lstm_baseline_clq_size = aggregate_lstm_data(lstm_baseline_data, "max_clique_sizes", aggregate=max)

print(lstm_baseline_clq_size)
print(lstm_clq_size)
print(bert_clq_size)

data_for_plot = [lstm_clq_size, bert_clq_size]
data_labels = ["LSTM", "Bert"]
multi_bar_plot(labels, data_for_plot, data_labels, 
               bar_y_label="#successful / #causal", 
               title="Maximum clique sizes for models trained on the hard dataset",
               bar_label_fmt='{}')

In [ ]:
low_nodes, values, high_nodes = arrange_data_by_bert_layer(bert_data, "max_clique_sizes", high_nodes=None, aggregate=max)
multi_line_plot(low_nodes, values, high_nodes, y_label="#successful / #causal", title="Max clique size by bert layer")

In [ ]:
low_nodes, values, high_nodes = arrange_data_by_bert_layer(bert_data, "avg_clique_sizes", high_nodes=None, aggregate=max)
multi_line_plot(low_nodes, values, high_nodes, y_label="#successful / #causal", title="Average clique size by bert layer")

In [ ]:
# Avg
labels, base_eq_rate, trained_eq_rate = arrange_data(rows, "avg_clique_sizes", aggregate=max, scale=1)
bar_plot(labels, x1=base_eq_rate, x2=trained_eq_rate, x1_label="Baseline", x2_label="Trained", 
         y_label="interv_eq_rate", title="Average clique sizes (max among all intervention locations)")

In [ ]:
# Sum
labels, base_eq_rate, trained_eq_rate = arrange_data(rows, "sum_clique_sizes", aggregate=max, scale=1)
bar_plot(labels, x1=base_eq_rate, x2=trained_eq_rate, x1_label="Baseline", x2_label="Trained", 
         y_label="interv_eq_rate", title="Sum of clique sizes (max among all intervention locations)")

## Max clique sizes, left vs right

In [ ]:
labels, _, left_max_clique = arrange_data(rows, "max_clique_sizes", aggregate=lambda s: s[0], scale=1)
labels, _, right_max_clique = arrange_data(rows, "max_clique_sizes", aggregate=lambda s: s[-1], scale=1)
bar_plot(labels, x1=left_max_clique, x2=right_max_clique, x1_label="Left", x2_label="Right", 
         y_label="Max clique sizes", title="Max clique sizes, left vs right", fmt="{}")

## Causal edge ratios

In [ ]:
labels, _, trained = arrange_data(rows, "causal_edge_ratios", aggregate=max, scale=100)
bar_plot(labels, x1=trained, x1_label="Trained", x1_color="orange",
         y_label="Causal edge ratio %", title="Percentage of edges that are causal within all cliques (max among all intervention locations)")

In [ ]:
labels, _, trained = arrange_data(rows, "max_clique_calusal_edge_ratios", aggregate=max, scale=100)
bar_plot(labels, x1=trained, x1_label="Trained", x1_color="orange",
         y_label="Causal edge ratio %", title="Percentage of edges that are causal within the largest clique (max among all intervention locations)")

In [ ]:
import numpy as np

def get_causal_edge_ratio(rows):
    labels = ["sentence_q", "subj_adj", "subj_noun", "subj", "neg", "negp", 
          "v_adv", "v_verb", "v_bar", "vp", "vp_q", 
          "obj_adj", "obj_noun", "obj"]
    labels2idx = {l: i for i, l in enumerate(labels)}
    trained = [0.0] * len(labels)
    for row in rows:
        abstraction = json.loads(row["abstraction"])
        high_node = abstraction[0]
        is_baseline = "baseline" in row["model_path"]
        if is_baseline: continue
        i = labels2idx[high_node]
        if col_name.endswith("s"):
            values = json.loads(row["max_clique_size"])
            
        else:
            value = row[col_name]
        
        value *= scale
        
        if is_baseline:

        trained[i] = value
            
    return labels, baseline, trained

## Verify and merge lstm-easy data

In [ ]:
# new_lstm_easy_data = db.select("experiment_data/lstm/lstm-easy.db", "results", cond_dict={"status": 1})
# old_lstm_easy_data = db.select("experiment_data/sep/nov2-500.db", "results", cond_dict={"status": 4}, like={"abstraction": "%lstm_0%"})

# numerator_cols = ["res_1_counts", "res_3_counts"]
# denominator_cols = ["res_0_counts", "res_1_counts", "res_2_counts", "res_3_counts"]
# labels, new_lstm_easy_interv_eq_rates = aggregate_lstm_data_by_ratio(new_lstm_easy_data, numerator_cols, denominator_cols, max)
# _, _, old_lstm_easy_interv_eq_rates = aggregate_lstm_data(old_lstm_easy_data, "interv_eq_rates", max)

# print(new_lstm_easy_interv_eq_rates)
# print(old_lstm_easy_interv_eq_rates)

# merge graph data into new lstm_easy

# for row in old_lstm_easy_data:
#     update_dict = {
#         "graph_save_paths": row["graph_save_paths"],
#         "max_clique_sizes": row["max_clique_sizes"],
#         "avg_clique_sizes": row["avg_clique_sizes"],
#         "sum_clique_sizes": row["sum_clique_sizes"],
#         "clique_counts": row["clique_counts"],
#         "visualize_save_paths": row["visualize_save_paths"]
#     }
#     db.add_cols("experiment_data/lstm/lstm-easy.db", "results", update_dict,)
#     db.update("experiment_data/lstm/lstm-easy.db", "results", update_dict, id=(row["id"]+1)//2)
